# imports

In [1]:
#!pip install --upgrade jinja2==3.0.1
#!pip install pixiedust
#!pip install torch_stft

#import pixiedust


In [2]:
from __future__ import print_function
import argparse
import pickle

import numpy as np
import torch
import torch.optim as optim
from gcommand_loader import GCommandLoader
from train import train, test
#from attacks import attack

import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
t = torch.cuda.get_device_properties(0).total_memory
print('t',t)
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print('a',a)
torch.cuda.empty_cache()
print('a',r)


t 11546525696
a 0
a 0


In [4]:
class args():

    def _init_(self, name):
        pass
    def cuda(self,x):
        pass         
    def train_path(self,x):
        pass
    def test_path(self,x):
        pass
    def test_batch_size(self,x):
        pass
    def arc(self,x):
        pass
    def epochs(self,x):
        pass
    def lr(self,x):
        pass
    def momentum(self,x):
        pass
    def optimizer(self,x):
        pass    
    def log_interval(self,x):
        pass
    def seed(self,x):
        pass
    def patience(self,x):
        pass
    def window_size(self,x):
        pass
    def window_stride(self,x):
        pass
    def window_type(self,x):
        pass    
    def normalize(self,x):
        pass
    def chkpt_path(self,x):
        pass
    def chkpt_path_train(self,x):
        pass
    def n_iter(self,x):
        pass
    def eps(self,x):
        pass    
    def alpha(self,x):
        pass
    def test_mode(self,x):
        pass

In [5]:
args.train_path='/home/data/gcommands/train'#
args.test_path='/home/data/gcommands/test'
args.valid_path='/home/data/gcommands/valid'
#args.train_path='/home/data/data2/valid'#/media/maayanl/HD-27/LPC/data/data2/
#args.test_path='/home/data/data2/test'
#args.valid_path='/home/data/data2/valid'
args.batch_size=128
args.test_batch_size=128
args.arc='VGG11'#'network architecture:  VGG16, VGG11, VGG13, VGG16, VGG19'
args.epochs=100#TODO:default=100

args.lr=0.001
args.momentum=0.9#help='SGD momentum, for SGD only'
args.optimizer='adam'#' help=optimization method: sgd | adam'
args.cuda=True
args.log_interval=50
args.cuda = args.cuda and torch.cuda.is_available()
args.seed="1434"
args.patience=5
args.window_size=0.02
args.window_stride=0.01
args.window_type='hamming'
args.normalize=True

args.test_mode=False #parser.add_argument('--test_mode', action="store_true", help='Whether to run model for test only or not')
args.chkpt_path="chkpt_path"
args.chkpt_path_train="chkpt_path_train"
args.n_iter=10
args.eps=0.01
args.alpha=0.01
import torch.nn as nn
import torch.nn.functional as F
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(174680, 1000)
        self.fc2 = nn.Linear(1000, 2)#TODO:WAS 35 run1

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
def _make_layers(cfg):
    layers = []
    in_channels = 1
    for x in cfg:
        if x == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                       nn.BatchNorm2d(x),
                       nn.ReLU(inplace=True)]
            in_channels = x
    layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
    return nn.Sequential(*layers)


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = _make_layers(cfg[vgg_name])
        self.fc1 = nn.Linear(7680, 512)
        self.fc2 = nn.Linear(512, 35)#TODO:WAS 35 run1

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return F.log_softmax(out,dim=1)


In [9]:
args.cuda = args.cuda and torch.cuda.is_available()
print('args.cuda' ,args.cuda )
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# build model
if args.arc == 'LeNet':
    model = LeNet()
elif args.arc.startswith('VGG'):
    model = VGG(args.arc)
else:
    model = LeNet()

if args.cuda:
    model = model.cuda()

# loading data
train_dataset = GCommandLoader(args.train_path, window_size=args.window_size, window_stride=args.window_stride,
                             window_type=args.window_type, normalize=args.normalize)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=True,
    num_workers=8, pin_memory=args.cuda, sampler=None)

valid_dataset = GCommandLoader(args.valid_path, window_size=args.window_size, window_stride=args.window_stride,
                             window_type=args.window_type, normalize=args.normalize)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=args.batch_size, shuffle=None,
    num_workers=8, pin_memory=args.cuda, sampler=None)

test_dataset = GCommandLoader(args.test_path, window_size=args.window_size, window_stride=args.window_stride,
                            window_type=args.window_type, normalize=True)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=args.test_batch_size, shuffle=None,
    num_workers=8, pin_memory=args.cuda, sampler=None)



# define optimizer
if args.optimizer.lower() == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
elif args.optimizer.lower() == 'sgd':
    optimizer = optim.SGD(model.parameters(), lr=args.lr,
                          momentum=args.momentum)
else:
    optimizer = optim.SGD(model.parameters(), lr=args.lr,
                          momentum=args.momentum)

best_valid_loss = np.inf
iteration = 0
epoch = 1



args.cuda True


In [11]:
if args.test_mode:
    checkpoint = torch.load(args.chkpt_path)
    model.load_state_dict(checkpoint['net'].state_dict())
    model.eval()

# train int with early stopping
while (epoch < args.epochs + 1) and (iteration < args.patience) and not args.test_mode:
    train(train_loader, model, optimizer, epoch, args.cuda, args.log_interval)
    valid_loss = test(valid_loader, model, args.cuda)
    plt.plot(valid_loss);

    if valid_loss > best_valid_loss:
        if epoch > 11:
           for param_group in optimizer.param_groups:
              param_group['lr'] = args.lr*0.2
        iteration += 1
        print('Loss was not improved, iteration {0}'.format(str(iteration)))
    else:
        print('Saving model...')
        iteration = 0
        best_valid_loss = valid_loss
        state = {
            'net': model,
            'acc': valid_loss,
            'epoch': epoch,
            'optimizer_state_dict:': optimizer.state_dict()
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, f'./checkpoint/{args.chkpt_path_train}.t7')
    epoch += 1


test(test_loader, model, args.cuda)
#attack(test_loader, 0, model, n_iter=10,eps=args.eps, alpha =args.alpha, rand_init=True)
plt.title("training loss");


Test set: Average loss: 22.6770, Accuracy: 204/392 (52%)



In [ ]:
import os

import torch
import librosa
import numpy as np
import numpy
import torch.nn.functional as F

def snr_db(x, desired_noise):
    return 20 * torch.log10(torch.sum(x ** 2) / torch.sum(desired_noise ** 2))

def relative_eps(snr):
    return torch.pow(torch.tensor(10.0), float(snr) / 20)

def pad_audio(audio, orig_size):
    max_size = (16000,)

    if audio.shape[1] < max_size[0]:
        torch.nn.functional.pad(audio,(0,max_size[0] - orig_size[1]))
    elif audio.shape[1] > max_size[0]:
        audio = audio[:max_size[0]]
    return audio
def spect_loader(padded_audio, normalize=True, max_len=101,args=args):
    window_size=args.window_size
    window_stride=args.window_stride
    window=args.window_type

    sr = 16000
    n_fft = int(sr * window_size)
    win_length = n_fft
    hop_length = int(sr * window_stride)

    n_fft = int(sr * window_size)
    win_length = n_fft
    hop_length = int(sr * window_stride)

    # STFT
    D = librosa.stft(padded_audio.detach().numpy(), n_fft=n_fft, hop_length=hop_length,
                     win_length=win_length, window=window)
    spect, phase = librosa.magphase(D)

    # S = log(S+1)
    spect = np.log1p(spect)
    spect=spect[0,:,:]
    # make all spects with the same dims
    # TODO: change that in the future
    if spect.shape[1] < max_len:
        pad = np.zeros((spect.shape[0], max_len - spect.shape[1]))
        spect = np.hstack((spect, pad))
    elif spect.shape[1] > max_len:
        spect = spect[:, :max_len]
    spect = np.resize(spect, (1, spect.shape[0], spect.shape[1]))
    spect = torch.FloatTensor(spect)

    # z-score normalization
    if normalize:
        mean = spect.mean()
        std = spect.std()
        if std != 0:
            spect.add_(-mean)
            spect.div_(std)
    return spect

In [1]:

def attack_wav(dataloader,args, x,y, model, n_iter, epsilon, alpha, rand_init_uniform=True,rand_init_normal=False, clip_min_max=True):
    model.eval()
    correct = 0
    total = 0
    y = torch.LongTensor([y]).cuda()
    audio, sr = librosa.load(x, sr=None)

    min_val, max_val = -1, 1 # audio.min(), audio.max()
    audio = np.expand_dims(audio, 0)

    audio = torch.FloatTensor(audio)
    spect=spect_loader(audio, normalize=True, max_len=101,args=args)
    spect = spect.unsqueeze(0).cuda()#.unsqueeze(0)
    delta = torch.zeros_like(spect, requires_grad=True)

    if rand_init_normal:
        delta.data = delta.data.normal_(std=1)
    elif rand_init_uniform:
        delta.data = delta.data.uniform_(-epsilon, epsilon)

    for i in range(n_iter):
        spect=spect+delta

        yhat = model(spect)
        loss = F.nll_loss(yhat, y)
        model.zero_grad()
        loss.backward()
        delta.data = delta.data + alpha * torch.sign(delta.grad.data)
        delta.data = torch.clamp(delta.data, min=-epsilon, max=epsilon)
        if clip_min_max:
            delta.data = torch.clamp(spect+delta.data, min=-min_val, max=max_val)-spect
        
    total += 1
    correct += int(yhat.argmax() != y.item())

    return spect+delta.detach(), sr, correct


def attack(dataloader,args, example_idx, model, n_iter=10, eps=0.01, alpha=0.01/2, rand_init=True):
    total=0
    for (x, y) in dataloader.dataset.spects:
        #x, y = dataloader.dataset.spects[example_idx]
        print(total)
        audio, sr = librosa.load(x, sr=16000)
        if(len(audio)<16000):
            print((x, y))

        else:
            adv_wav, sr ,cr= attack_wav(dataloader,args, x, y, model, n_iter, eps, alpha, rand_init,True,example_idx)
            cr+=cr
            total += 1
    print(f"PGD accuracy {cr*100/total} ({cr}/{total})")
    return adv_wav

attack(test_loader,args,2, model, n_iter=10,eps=args.eps, alpha =args.alpha, rand_init=True)

NameError: name 'test_loader' is not defined

In [ ]:
import IPython.display as ipd

x, y = test_loader.dataset.spects[0]
audio, sr = librosa.load(x, sr=8000)
ipd.Audio(audio, rate=sr)


In [ ]:
checkpoint=torch.load('./checkpoint/T35_82.t7')